In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("loop.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [3]:
def filter_by_normal(pcd):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*0.01
            pass
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [4]:
def compute_transform(vector1, vector2):
    # Normalize the vectors to unit length
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Compute the rotation matrix
    rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()

    # Compute the translation vector
    translation_vector = vector2 - np.dot(rotation_matrix, vector1)

    return translation_vector, rotation_matrix

In [7]:
def create_voxelgird_from_list(point_coordinates, voxel_size):
    pc = o3d.geometry.PointCloud()
    
    pc.points.extend(o3d.utility.Vector3dVector(point_coordinates))
    
    vg = o3d.geometry.VoxelGrid.create_from_point_cloud(pc, voxel_size)
    
    return vg
    

In [8]:
def is_array_in_list(target_array, array_list):
    for array in array_list:
        if np.array_equal(target_array, array):
            return True
    return False

In [9]:
def create_uniform_pc_from_bb(bb_axis, voxel_size, color):
    pc = o3d.geometry.PointCloud()
    bb = o3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(bb_axis)
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(bb)
    mesh = mesh.paint_uniform_color([0,1,0])
    vg = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size)
    voxels = vg.get_voxels()
    grid_indexes = [x.grid_index for x in voxels]
    
    voxel_centers = [vg.get_voxel_center_coordinate(index) for index in grid_indexes]
    
    pc.points.extend(o3d.utility.Vector3dVector(voxel_centers))  
    pc.paint_uniform_color(color)
    return pc



In [10]:
bb = pcd_flat.get_axis_aligned_bounding_box()
bb.color = np.asarray([0,0,1])
uniform_pc = create_uniform_pc_from_bb(bb, 0.1, [1,0,0])
o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

In [11]:
def find_closest_vector_index(input_vector, pc):
    # Convert the input vector and vector list to numpy arrays for easier calculations
    input_vector = np.array(input_vector)
    vector_list = np.asarray(pc.points)

    # Calculate the Euclidean distances between the input vector and all vectors in the list
    distances = np.linalg.norm(vector_list - input_vector, axis=1)

    # Find the index of the vector with the minimum distance
    closest_index = np.argmin(distances)

    return closest_index

In [17]:
#pcd_tree = o3d.geometry.KDTreeFlann(uniform_pc)
#uniform_pc = uniform_pc.paint_uniform_color([1,0,0])
#
#o3d.visualization.draw_geometries([pcd_flat, uniform_pc])
#
#
#
#
#print("Find its neighbors with distance less than 0.2, and paint them green.")
#
#search_radius = np.ceil(np.sqrt(0.1**2+0.1**2)*10000)/10000
#
#index = find_closest_vector_index([-6.5,0,1], uniform_pc)
#
#
#[k, idx, _] = pcd_tree.search_radius_vector_3d(uniform_pc.points[index], search_radius)
#np.asarray(uniform_pc.colors)[idx[1:], :] = [0, 1, 0]
#
#new_neighbours = list(idx[1:])
#old_set = set([])
#new_set = set(new_neighbours)
#
#tmp = []
#for x in range(50):
#    
#    color = np.random.rand(3)
#    
#    
#    different_elements = old_set.symmetric_difference(new_set)
#    new_neighbours = list(different_elements)
#    old_set = new_set
#    
#    for index in new_neighbours:
#        [k, idx, _] = pcd_tree.search_radius_vector_3d(uniform_pc.points[index], search_radius)
#        np.asarray(uniform_pc.colors)[idx[1:], :] = color
#        tmp.extend(idx[1:])
#    
#    new_set = set(tmp)
#    
#    #test_pc = test_pc.select_by_index(tmp2, invert=True)
#    
#
#    o3d.visualization.draw_geometries([uniform_pc, pcd_flat])




Find its neighbors with distance less than 0.2, and paint them green.


In [12]:
## TEST: find neighbours from one pointcloud in other pointcloud
#index = find_closest_vector_index([-6.5,0,0], uniform_pc)
#
#uniform_pc.paint_uniform_color([1,0,0])
#pcd_flat.paint_uniform_color([0,0,0])
#
#uniform_tree = o3d.geometry.KDTreeFlann(uniform_pc)
#flat_tree = o3d.geometry.KDTreeFlann(pcd_flat)
#
#
#[k, idxu, _] = uniform_tree.search_radius_vector_3d(uniform_pc.points[index], 4)
#np.asarray(uniform_pc.colors)[idxu[1:], :] = [0, 1, 0]
#[k, idxf, _] = flat_tree.search_radius_vector_3d(uniform_pc.points[index], 4)
#np.asarray(pcd_flat.colors)[idxf[1:], :] = [0, 1, 0]
#
#o3d.visualization.draw_geometries([uniform_pc, pcd_flat])


In [13]:
def knn_search_pointclouds(pc_grid, pc_target, center, radius):
    index = find_closest_vector_index(center, pc_grid)
    
    
    tree_grid = o3d.geometry.KDTreeFlann(pc_grid)
    tree_target = o3d.geometry.KDTreeFlann(pc_target)


    [ka, idxa, _] = tree_grid.search_radius_vector_3d(pc_grid.points[index], radius)
    
    [kb, idxb, _] = tree_target.search_radius_vector_3d(pc_grid.points[index], radius)
    
    return ka, kb, idxa, idxb
    

In [15]:
def get_ids_in_direction(idx_grid, idx_target, coords_grid, coords_target, center, direction):
    
   

    d = {
    "up": (1,False),
    "down": (1,True),
    "left": (0,True),
    "right": (0,False)}
    
    xory = d[direction][0]
    smaller = d[direction][1]
    
    idx_dir_grid = []
    idx_dir_target = []
    for i in range(len(idx_grid)):
        if smaller:
            if coords_grid[i][xory] < center[xory]:
                idx_dir_grid.append(idx_grid[i])
        else:
            if coords_grid[i][xory] > center[xory]:
                idx_dir_grid.append(idx_grid[i])
    
    for i in range(len(idx_target)):
        if smaller:
            if coords_target[i][xory] < center[xory]:
                idx_dir_target.append(idx_target[i])
        else:
            if coords_target[i][xory] > center[xory]:
                idx_dir_target.append(idx_target[i])
            
            
    return (idx_dir_grid, idx_dir_target)
        

In [16]:
def split_helper(data):
    idx, coords, center = data
    
    up = []
    down = []
    left = []
    right = []
    
    
    x = coords[0]
    y = coords[1]
    x_center = center[0]
    y_center = center[1]
    if (x_center < x):
        right = idx
    if (x_center > x):
        left = idx
    if (y_center < y):
        up = idx
    if (y_center > y):
        down = idx
    
    return up, down, left, right


def split_ids_into_directions2(idx_grid, idx_target, coords_grid, coords_target, center):
    
    idx_grid = np.asarray(idx_grid) 
    idx_target = np.asarray(idx_target)

    
    process_data = zip(idx_grid, coords_grid, [center]*len(idx_grid))
    with Pool(8) as p:
            
        res_grid = p.map(split_helper, process_data)
    #res_grid = np.asarray(res_grid)
    
    print(res_grid)
    sleep(1)

    up_grid = [res[0] for res in res_grid if not isinstance(res[0], list)]
    down_grid = [res[1] for res in res_grid if not isinstance(res[1], list)]
    left_grid = [res[2] for res in res_grid if not isinstance(res[2], list)]
    right_grid = [res[3] for res in res_grid if not isinstance(res[3], list)]

   
    process_data = zip(idx_target, coords_target, [center]*len(idx_target))
    with Pool(8) as p:
            
        res_target = p.map(split_helper, process_data)

    up_target = [res[0] for res in res_target if not isinstance(res[0], list)]
    down_target = [res[1] for res in res_target if not isinstance(res[1], list)]
    left_target = [res[2] for res in res_target if not isinstance(res[2], list)]
    right_target = [res[3] for res in res_target if not isinstance(res[3], list)]
    
    
          
    
   
    return (up_grid, down_grid, left_grid, right_grid, up_target, down_target, left_target, right_target)



In [17]:
def split_ids_into_directions(idx_grid, idx_target, coords_grid, coords_target, center):
    up_grid = []
    down_grid = []
    left_grid = []
    right_grid = []

    up_target = []
    down_target = []
    left_target = []
    right_target = []

    for i in range(len(idx_grid)):
        x_grid = coords_grid[i][0]
        y_grid = coords_grid[i][1]
        x_center = center[0]
        y_center = center[1]
        if (x_center < x_grid):
            right_grid.append(idx_grid[i])
        if (x_center > x_grid):
            left_grid.append(idx_grid[i])
        if (y_center < y_grid):
            up_grid.append(idx_grid[i])
        if (y_center > y_grid):
            down_grid.append(idx_grid[i])

    for i in range(len(idx_target)):
        x_target = coords_target[i][0]
        y_target = coords_target[i][1]
        x_center = center[0]
        y_center = center[1]
        if (x_center < x_target):
            right_target.append(idx_target[i])
        if (x_center > x_target):
            left_target.append(idx_target[i])
        if (y_center < y_target):
            up_target.append(idx_target[i])
        if (y_center > y_target):
            down_target.append(idx_target[i])        

    
    return (up_grid, down_grid, left_grid, right_grid, up_target, down_target, left_target, right_target)



In [29]:
# search multiple iterations until boundary found with multiprocessing: currently worse performance
search_radius = np.ceil(np.sqrt(0.1**2+0.1**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

#initial search
k_grid, k_target, idxu, idxf = knn_search_pointclouds(uniform_pc, pcd_flat, [-6.5,0,0], search_radius)
np.asarray(uniform_pc.colors)[idxu[1:], :] = [0, 0, 1]

new_neighbours = list(idxu[1:])
old_set = set([])
new_set = set(new_neighbours)



threshold = 2

tmp = []
targets_hit = 0
while(False):
    
    color = [0, 0, 1]
    
    
    different_elements = old_set.symmetric_difference(new_set)
    new_neighbours = list(different_elements)
    old_set = new_set
    
    for index in new_neighbours:
        k_grid, k_target, idxu, idxf = knn_search_pointclouds(uniform_pc, pcd_flat, uniform_pc.points[index], search_radius)
        
        coords_grid = np.asarray(uniform_pc.points)[idxu]
        coords_target = np.asarray(pcd_flat.points)[idxf]
        
        
        
        
        directions = ["up", "down", "left", "right"]
        idxu = np.asarray(idxu)
        idxf = np.asarray(idxf)
        fixed_args = [idxu,idxf,coords_grid, coords_target, uniform_pc.points[index]]
        #args_list1 = [(fixed_args + (arg,)) for arg in directions]
        args_list = [fixed_args + [arg] for arg in directions]
        
        
        with Pool(4) as p:
            
            res = p.starmap(get_ids_in_direction, args_list)

        
        idx_up_grid, idx_up_target = res[0]
        idx_down_grid, idx_down_target = res[1]
        idx_left_grid, idx_left_target = res[2]
        idx_right_grid, idx_right_target = res[3]
        
        
        
        


        #idx_up_grid, idx_up_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, uniform_pc.points[index], "up")
        #idx_down_grid, idx_down_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, uniform_pc.points[index], "down")
        #idx_left_grid, idx_left_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, uniform_pc.points[index], "left")
        #idx_right_grid, idx_right_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, uniform_pc.points[index], "right")
        
        if (len(idx_up_target) < threshold):
            tmp.extend(idx_up_grid[1:])
            np.asarray(uniform_pc.colors)[idx_up_grid[1:], :] = color
        
        if (len(idx_down_target) < threshold):
            tmp.extend(idx_down_grid[1:])
            np.asarray(uniform_pc.colors)[idx_down_grid[1:], :] = color
        
        if (len(idx_left_target) < threshold):
            tmp.extend(idx_left_grid[1:])
            np.asarray(uniform_pc.colors)[idx_left_grid[1:], :] = color
        
        if (len(idx_right_target) < threshold):
            tmp.extend(idx_right_grid[1:])
            np.asarray(uniform_pc.colors)[idx_right_grid[1:], :] = color

        
         
    targets_hit += 1
    print(targets_hit)
    new_set = set(tmp)
    if (targets_hit % 5 == 0):
        o3d.visualization.draw_geometries([uniform_pc, pcd_flat])
    #test_pc = test_pc.select_by_index(tmp2, invert=True)
    #rint(targets_hit)



In [245]:
# search multiple iterations until boundary found
search_radius = np.ceil(np.sqrt(0.2**2+0.2**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

PointCloud with 36217 points.

In [33]:


#initial search
k_grid, k_target, idxu, idxf = knn_search_pointclouds(uniform_pc, pcd_flat, [-6.5,0,0], search_radius)
np.asarray(uniform_pc.colors)[idxu[1:], :] = [0, 0, 1]

#new_neighbours = list(idxu[1:])
old_set = set()
#new_set = set(new_neighbours)
new_set = set(idxu[1:])

print(new_set)
threshold = 2

tmp = set()
targets_hit = 0

different_elements = old_set.symmetric_difference(new_set)
while(different_elements):
    
    color = [0, 0, 1]
    
    
    different_elements = old_set.symmetric_difference(new_set)
    #new_neighbours = list(different_elements)
    old_set = new_set
    
    for index in different_elements:
        new_center = uniform_pc.points[index]
        k_grid, k_target, idxu, idxf = knn_search_pointclouds(uniform_pc, pcd_flat, new_center, search_radius)
        
        coords_grid = np.asarray(uniform_pc.points)[idxu]
        coords_target = np.asarray(pcd_flat.points)[idxf]

        
        res = split_ids_into_directions(idxu, idxf, coords_grid, coords_target, new_center)

        idx_up_grid = res[0]
        idx_down_grid = res[1]
        idx_left_grid = res[2]
        idx_right_grid = res[3]
        idx_up_target = res[4]
        idx_down_target = res[5]
        idx_left_target = res[6]
        idx_right_target = res[7]
        
        
       

        if (len(idx_up_target) < threshold):
            tmp.update(idx_up_grid[1:])
            np.asarray(uniform_pc.colors)[idx_up_grid[1:], :] = color
        
        if (len(idx_down_target) < threshold):
            tmp.update(idx_down_grid[1:])
            np.asarray(uniform_pc.colors)[idx_down_grid[1:], :] = color
        
        if (len(idx_left_target) < threshold):
            tmp.update(idx_left_grid[1:])
            np.asarray(uniform_pc.colors)[idx_left_grid[1:], :] = color
        
        if (len(idx_right_target) < threshold):
            tmp.update(idx_right_grid[1:])
            np.asarray(uniform_pc.colors)[idx_right_grid[1:], :] = color

        
        
    targets_hit += 1
    
    new_set = set(tmp)
    #if (targets_hit % 20 == 0):
        #o3d.visualization.draw_geometries([uniform_pc, pcd_flat])
    #test_pc = test_pc.select_by_index(tmp2, invert=True)
    #rint(targets_hit)



{16128, 17409, 17411, 9998, 16019, 3476, 17303, 13593, 13984, 16545, 15401, 17583, 10287, 6575, 17587, 12469, 3769, 3920, 17491, 14683, 17261, 17265, 17267, 11380}


In [34]:
o3d.visualization.draw_geometries([uniform_pc, pcd_flat])

In [249]:
def grow_void(pcd_grid, pcd_target, initial_seed, initial_set=set(), search_radius=0.2, stop_threshold=2, color=[0,0,1]):

    #initial search
    
    k_grid, k_target, idxu, idxf = knn_search_pointclouds(pcd_grid, pcd_target, initial_seed, search_radius)
    np.asarray(pcd_grid.colors)[idxu[1:], :] = color
    
    
    old_set = set()
    
    new_set = set(idxu[1:])
    different_elements = new_set-old_set-initial_set
    old_set = set(new_set) 
    
    while(different_elements):
    
                     
        
        
        for index in different_elements:
            new_center = pcd_grid.points[index]
            k_grid, k_target, idxu, idxf = knn_search_pointclouds(pcd_grid, pcd_target, new_center, search_radius)
            
            coords_grid = np.asarray(pcd_grid.points)[idxu]
            coords_target = np.asarray(pcd_target.points)[idxf]

            

            idx_up_grid, idx_up_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "up")
            idx_down_grid, idx_down_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "down")
            idx_left_grid, idx_left_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "left")
            idx_right_grid, idx_right_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "right")
            
            if (len(idx_up_target) < stop_threshold):
                new_set.update(idx_up_grid[1:])
                np.asarray(pcd_grid.colors)[idx_up_grid[1:], :] = color
            
            
            if (len(idx_down_target) < stop_threshold):
                new_set.update(idx_down_grid[1:])
                np.asarray(pcd_grid.colors)[idx_down_grid[1:], :] = color
            
            if (len(idx_left_target) < stop_threshold):
                new_set.update(idx_left_grid[1:])
                np.asarray(pcd_grid.colors)[idx_left_grid[1:], :] = color
            
            if (len(idx_right_target) < stop_threshold):
                new_set.update(idx_right_grid[1:])
                np.asarray(pcd_grid.colors)[idx_right_grid[1:], :] = color
        
        different_elements = new_set-old_set-initial_set
        old_set = set(new_set) 
        
    new_set = new_set.union(initial_set)
    
    return pcd_grid, new_set
        



In [250]:
known_points = set()
res = grow_void(uniform_pc, pcd_flat, [-6.5,0,0], color=[0,0,1])
scan_positions = res[0]
known_points = res[1]
res = grow_void(uniform_pc, pcd_flat, [-8,0,0], color=[0,1,1], initial_set=known_points)
scan_positions = res[0]
known_points = res[1]
res = grow_void(uniform_pc, pcd_flat, [-10,0,0], color=[0,1,0], initial_set=known_points)
scan_positions = res[0]
known_points = res[1]

#o3d.visualization.draw_geometries([scan_positions])

In [236]:
known_points = set()
res = grow_void(uniform_pc, pcd_flat, [-6.5,0,0], color=[0,0,1])
scan_positions = res[0]
known_points = set()
res = grow_void(uniform_pc, pcd_flat, [-8,0,0], color=[0,1,1], initial_set=known_points)
scan_positions = res[0]
known_points = set()
res = grow_void(uniform_pc, pcd_flat, [-10,0,0], color=[0,1,0], initial_set=known_points)
scan_positions = res[0]
known_points = set()

In [251]:
o3d.visualization.draw_geometries([scan_positions, pcd_flat])